In [25]:
import numpy as np
import pandas as pd
import csv
import itertools
import torch
from torch.utils.data import DataLoader, Dataset
import pickle
import dataset_methods

In [26]:
games = pd.read_csv("games.csv")
games_df = pd.DataFrame(games)
print(games_df.columns)
print(len(games_df))

Index(['id', 'rated', 'created_at', 'last_move_at', 'turns', 'victory_status',
       'winner', 'increment_code', 'white_id', 'white_rating', 'black_id',
       'black_rating', 'moves', 'opening_eco', 'opening_name', 'opening_ply'],
      dtype='object')
20058


In [27]:
def mod_dataset(games, min_turns, max_turns):
    
    games = games.drop(['victory_status', 'opening_name', 'id', 'white_id', 'black_id', 'rated', 'created_at',
                    'last_move_at', 'increment_code', 'white_rating', 'black_rating', 'opening_eco',
                    'opening_ply'], axis=1)
    
    games = games.query(f"turns >={min_turns}")
    games = games.query(f"turns <={max_turns}")
    games = games.query("winner != 'draw'")
    
    # now we split the 'moves' column into a list of individual moves:
    games['moves']=games['moves'].str.split()
    
    max_len = games['turns'].max()
    print(f"max_len={max_len}")

    # Define a lambda function to pad each list of strings
    pad_moves = lambda moves_list: moves_list + [' '] * (max_len - len(moves_list))

    # Apply the lambda function to the 'moves' column of the games DataFrame
    games['moves'] = games['moves'].apply(pad_moves)
    
    # reset the index
    games = games.reset_index()

    # drop the old index column
    games = games.drop('index', axis=1)
    
    return games

def print_stats(dataset):
    print(f"largest: {dataset['turns'].nlargest(1)},\nsmallest: {dataset['turns'].nsmallest(1)}")
    print(f"dataset length: {len(dataset)}")
    print(dataset['moves'][0]) # all the moves in a turn
    print(len(dataset['moves'][0]))
    print(dataset['turns'][0])
    print(dataset.iloc[0:20])
    
    print(f"first move: {dataset['moves'][0]}")
    print(f"last move: {dataset['moves'][len(dataset)-1]}")
    
def dump_dataset(dataset, string):
    pickle.dump(dataset, open(f"{string}.pkl", "wb")) # games3 has padded strings for 'moves'
    

In [28]:
dataset_50 = mod_dataset(games_df, min_turns = 5, max_turns = 50)
dump_dataset(dataset_50, "games_50")

max_len=50


/var/folders/lh/6svpwf5n6zzg8m2whfkn1n2h0000gn/T/ipykernel_71145/3422649068.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games['moves']=games['moves'].str.split()
/var/folders/lh/6svpwf5n6zzg8m2whfkn1n2h0000gn/T/ipykernel_71145/3422649068.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games['moves'] = games['moves'].apply(pad_moves)


In [6]:
dataset_20 = mod_dataset(games_df, min_turns = 5, max_turns = 20)
dump_dataset(dataset_20, "games_20")
#print_stats(dataset_20)

max_len=20


/var/folders/lh/6svpwf5n6zzg8m2whfkn1n2h0000gn/T/ipykernel_71145/3422649068.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games['moves']=games['moves'].str.split()
/var/folders/lh/6svpwf5n6zzg8m2whfkn1n2h0000gn/T/ipykernel_71145/3422649068.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games['moves'] = games['moves'].apply(pad_moves)


In [17]:
dataset_100 = mod_dataset(games_df, min_turns = 5, max_turns = 100)
dump_dataset(dataset_100, "games_100")
#print_stats(dataset_100)

max_len=100


/var/folders/lh/6svpwf5n6zzg8m2whfkn1n2h0000gn/T/ipykernel_71145/3422649068.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games['moves']=games['moves'].str.split()
/var/folders/lh/6svpwf5n6zzg8m2whfkn1n2h0000gn/T/ipykernel_71145/3422649068.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games['moves'] = games['moves'].apply(pad_moves)


In [7]:
len(dataset_20)

1385

In [18]:
len(dataset_100)

16631

In [10]:
test, train = dataset_methods.get_dataset("games_20.pkl")

In [11]:
len(test)

1108

In [12]:
len(train)

277

In [15]:
len(train) / len(test)

0.25

In [19]:
test_, train_ = dataset_methods.get_dataset("games_100.pkl")

In [21]:
print(len(train_))
print(len(test_))

3326
13305


In [22]:
len(train_) / len(test_)

0.24998121007140173